# Leetcode
---
Preparing tech interviews in the short run, take the programming skills to the next level in the long run.
[Website](https://leetcode.com)


# Two sum (#1)
[Exercise](https://leetcode.com/problems/two-sum)

Given an array of integers, return indices of the two numbers such that they add up to a specific target.

You may assume that each input would have **exactly** one solution, and you may not use the same element twice.

Example:

Given `nums = [2, 7, 11, 15], target = 9`

Because nums[0] + nums[1] = 2 + 7 = 9,  
`return [0, 1]`



In [1]:
def assertions():
    # define checks: list, target, expected output
    checks = (
        ([-3, 4, 3, 90], 0, [0, 2]),
        ([2, 7, 11, 15], 9, [0, 1]),
        ([0, 0, 4, 7], 0, [0, 1]),
        ([3, 2, 4], 6, [1, 2]),
        ([-1, -2, -3, -4, -5], -8, [2, 4]),
    )
    
    failed = False
    for c in checks:
        E = twoSum(c[0], c[1])
        if E != c[2]:
            print(c[0], c[1], E, c[2])
            failed = True
            break
    if failed:
        return

## Numpy approach
* Convert the list into a numpy array to efficiently iterate. 
* Then, for each integer in the array calculate the sum of each element with 
  that integer to check whether any sum adds to the target (winner). 
* Then search those positions in the array, first, the current iteration 
  element.
* Chances are the location returns two positions if the target is the sum
  of the same numbers in different positions in the array. 
* Otherwise, search the position of the second element.   

In [2]:
import numpy as np
def twoSum(nums, target):
    """Return the indices of the integers that sum up to the target."""
    array = np.array(nums)
    for n, i in enumerate(array):
        if (array[n+1:] + i == target).any():
            # There is a winner in this iteration
            first_pos = np.where(array == i)[0]
            if first_pos.size == 2:
                return [first_pos[0], first_pos[1]]
            return [
                first_pos[0], np.where(array == target - i)[0][0]]

In [3]:
%%timeit
assertions()

65.5 µs ± 1.29 µs per loop (mean ± std. dev. of 7 runs, 10000 loops each)


## Hash table approach (2-loops)
We need a way to get the index of the conjugate and that could be achieved by usign a `dict` where:
* Keys are the values we'll search the conjugate within.
* Values are the current conjugate position
* For any number in the array we'll calculate the conjugate and search in the keys

In [4]:
def twoSum(nums, target):
    indices = {n: idx for idx, n in enumerate(nums)}
    for idx, n in enumerate(nums):
        conjugate = target - n
        if conjugate in indices.keys() and indices[conjugate] != idx:
            return [idx, indices[conjugate]]

In [5]:
%%timeit
assertions()

5.5 µs ± 87 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Hash table approach (1-loop)
Update the dict on the fly and look back for the conjugate

In [6]:
def twoSum(nums, target):
    indices = {}
    for idx, n in enumerate(nums):
        conjugate = target - n
        if conjugate in indices.keys():
            return [indices[conjugate], idx]
        indices.update([[n, idx], [n, idx]])

In [7]:
%%timeit
assertions()

6.58 µs ± 33.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Numpy matrix approach
Create a matrix out of the sum of the arrays and look for the target's position. Notice that the matrix created is diagonal so usually `np.where` will return a couple of indexers --the rows and the columms-- with two numbers inside meaning x, y coordinates. In the case of `[0, 0, 4, 7]`, four coordinates will be returned meaning the sum of the submatrix `[[0, 0],[0, 0]` in the top left corner.

This approach greatly exceeded the time limit processing a quite big array.

In [8]:
def twoSum(nums, target):
    n = len(nums)
    matrix = np.full((n, n), nums)
    diagonal = matrix + matrix.T
    # exclude diagonal from the results
    diagonal[np.eye(n).astype(bool)] = target + 1
    find = np.where(diagonal == target)
    if find[0].size == 2:
        return [find[0][0], find[0][1]]
    else:
        return [find[0][1], find[0][2]]

In [9]:
%%timeit
assertions()

70 µs ± 860 ns per loop (mean ± std. dev. of 7 runs, 10000 loops each)


# Running Sum of 1d Array (#1480)

**[Exercise](https://leetcode.com/problems/running-sum-of-1d-array/)**

Given an array `nums`. We define a running sum of an array as `runningSum[i] = sum(nums[0]…nums[i])`.

Return the running sum of `nums`.

 

Example 1:

Input: nums = [1,2,3,4]  
Output: [1,3,6,10]  
Explanation: Running sum is obtained as follows: [1, 1+2, 1+2+3, 1+2+3+4].  

Example 2:

Input: nums = [1,1,1,1,1]  
Output: [1,2,3,4,5]  
Explanation: Running sum is obtained as follows: [1, 1+1, 1+1+1, 1+1+1+1, 1+1+1+1+1].  

Example 3:

Input: nums = [3,1,2,10,1]  
Output: [3,4,6,16,17]  

 

Constraints:

* 1 <= nums.length <= 1000
* -10^6 <= nums[i] <= 10^6

In [1]:
import numpy as np
def assertions():
    # define checks: list, target, expected output
    checks = (
        ([1, 1, 1, 1], [1, 2, 3, 4,]),
        ([1, 10, 20, 7], [1, 11, 31, 38]),
        ([1, 2, 3, 4], [1, 3, 6, 10])
    )
    
    failed = False
    for c in checks:
        E = runningSum(c[0])
        if E != c[1]:
            print('Expected array for {}: {}; returned: {}'.format(
                   c[0], c[1], E,))
            failed = True
            break
    if failed:
        return

## Numpy.cumsum() approach
Maybe the shortest and easiest (but not the fastest) way to achieve it is the numpy's `cumsum()` method

In [2]:
def runningSum(nums):
    return np.array(nums).cumsum().tolist()

In [3]:
%%timeit
assertions()

6.46 µs ± 115 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


## Simple Iterator approach

In [4]:
def runningSum(nums):
    output = nums[:1]
    for number in nums[1:]:
        output.append(output[-1] + number)
    return output

In [5]:
%%timeit
assertions()

1.78 µs ± 4 ns per loop (mean ± std. dev. of 7 runs, 1000000 loops each)


A more succint, yet slower, one

In [6]:
def runningSum(nums):
    return [sum(nums[:idx+1]) for idx, _ in enumerate(nums)]

In [7]:
%%timeit
assertions()

3.2 µs ± 42.5 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


In [8]:
def runningSums(nums):
    for idx, number in enumerate(nums):
        nums[idx] += nums[idx-1]
    return nums

In [9]:
%%timeit
assertions()

3.21 µs ± 22.7 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)


# Reverse integer

**[Exercise](https://leetcode.com/problems/reverse-integer/)**

Given a 32-bit signed integer, reverse digits of an integer.

Example 1:

Input: 123  
Output: 321

Example 2:

Input: -123  
Output: -321

Example 3:

Input: 120  
Output: 21

Note:  
Assume we are dealing with an environment which could only store integers within the 32-bit signed integer range: $[−2^{31},  2^{31} − 1]$. For the purpose of this problem, assume that your function returns 0 when the reversed integer overflows.


In [36]:
import numpy as np
def assertions():
    # define checks: input integer and expected output
    checks = (
        (123, 321),
        (-123, -321),
        (120, 21),
        (214748364, 463847412),
        (2147483648, 0)  # Overflow
    )
    
    failed = False
    for c in checks:
        E = reverse(c[0])
        if E != c[1]:
            print('Expected integer for {}: {}; returned: {}'.format(
                   c[0], c[1], E,))
            failed = True
            break
    if failed:
        return

## Exponentiation approach
Multiply each digit in the int by growing powers of ten  

$123 = 1\times 10^2+2\times 10^1+3\times 10^0$  
$321 = 1\times 10^0+2\times 10^1+3\times 10^2$

It ranked quite nice, over 73% 

In [44]:
def reverse(i):
    # get the digits excluding the sign
    negative = i < 0
    digits = str(i)[1:] if negative else str(i)
    outcome = 0
    for e, d in enumerate(digits):
        outcome += int(d)*10**e
    
    # rescue the sign (if any)
    if negative:
        outcome *= -1
    
    # Overflows int32
    if outcome < -2**31 or outcome > 2**31-1:
        outcome = 0
        
    return outcome


In [45]:
%%timeit
assertions()

10.2 µs ± 48.8 ns per loop (mean ± std. dev. of 7 runs, 100000 loops each)
